# Libraries

In [1]:
import pandas as pd
import numpy as np

# Loading results

In [ ]:
result_rafael = pd.read_csv('results_rafael.csv')
result_10_rafael = pd.read_csv('results_10_rafael.csv')

result_luiz = pd.read_csv('results_luiz.csv')
result_10_luiz = pd.read_csv('results_10_luiz.csv')

# result_rafael = pd.concat([result_rafael, result_10_rafael])
# result_luiz = pd.concat([result_luiz, result_10_luiz])

result_10_rafael = result_rafael[result_rafael['Qnt'] != 'DySyn-TS']
result_10_luiz = result_luiz[result_luiz['Qnt'] != 'DySyn-TS']

# Ensure the 'MAE' column is numeric
result_rafael['MAE'] = pd.to_numeric(result_rafael['MAE'], errors='coerce')
result_luiz['MAE'] = pd.to_numeric(result_luiz['MAE'], errors='coerce')

grouped_rafael = result_rafael.groupby(['Qnt', 'MFte', 'MFtr']).agg({'MAE': 'mean'}).reset_index()
grouped_luiz = result_luiz.groupby(['Qnt', 'MFte', 'MFtr']).agg({'MAE': 'mean'}).reset_index()

grouped = pd.concat([grouped_rafael, grouped_luiz])

C:\Users\joaop\AppData\Local\Temp\ipykernel_11116\1831204099.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  result_10_luiz = pd.read_csv('results_10_luiz.csv')


# Plots

## Line Plot

In [3]:
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

def line_plot(grouped):
    mftr_dropdown = widgets.Dropdown(
        options=grouped['MFtr'].unique(),
        description='MFtr:'
    )

    mftr_dropdown.value = grouped['MFtr'].unique()[0]

    def update_plot(change):
        selected_mftr = mftr_dropdown.value
        filtered_data = grouped[grouped['MFtr'] == selected_mftr]

        fig = go.Figure()
        for qnt_val in filtered_data['Qnt'].unique():
            qnt_data = filtered_data[filtered_data['Qnt'] == qnt_val]
            fig.add_trace(go.Scatter(
                x=qnt_data['MFte'], 
                y=qnt_data['MAE'], 
                mode='lines+markers', 
                name=qnt_val,
                visible='legendonly'  # Set initial visibility to legendonly
            ))

        fig.update_layout(title=f'MAE vs MFte for MFtr = {selected_mftr}',
                          xaxis_title='MFte',
                          yaxis_title='MAE')

        clear_output(wait=True)
        display(fig)
        display(mftr_dropdown)

    mftr_dropdown.observe(update_plot, names='value')
    display(mftr_dropdown)
    update_plot(None)

In [ ]:
line_plot(grouped_rafael)

Dropdown(description='MFtr:', index=9, options=(np.float64(0.05), np.float64(0.15), np.float64(0.25), np.float…

In [5]:
line_plot(grouped_luiz)

Dropdown(description='MFtr:', index=3, options=(0.05, 0.25, 0.35, 0.45, 0.55, 0.65, 0.95, '0.05', '0.15', '0.2…

In [6]:
line_plot(grouped)

Dropdown(description='MFtr:', index=6, options=(0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, '0…